In [1]:
import MeCab
import time
import random
import numpy as np
import time
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
class Vocab(object):
    """語彙を管理するためのクラス"""
    def __init__(self, word2id={}):
        """
        :param word2id: 単語(str)をインデックス(int)に変換する辞書
        """
        self.word2id = dict(word2id)
        self.id2word = {v: k for k, v in self.word2id.items()}    

    def build_vocab(self, sentences, min_count=1):
        """コーパスから語彙の辞書を構築するメソッド

        :param sentences: list of list of str, コーパス
        :param min_count: int, 辞書に含める単語の最小出現回数
        """
        # 各単語の出現回数をカウントする辞書を作成します
        word_counter = {}
        for sentence in sentences:
            for word in sentence:
                # dict.get(key, 0)はdictにkeyがあればdict[key]を、なければ0を返すメソッドです
                word_counter[word] = word_counter.get(word, 0) + 1

        # min_count回以上出現する単語のみ語彙に加えます
        # 出現回数の高い単語から順にword2idに追加していきます
        # 出現回数に-1をかけた値でsortすることで出現回数の降順になるようにしています
        for word, count in sorted(word_counter.items(), key=lambda x: -x[1]):
            if count < min_count:
                break
            _id = len(self.word2id)
            self.word2id.setdefault(word, _id)
            self.id2word[_id] = word

        # 語彙に含まれる単語の出現回数を保持します（あとで使います）
        self.raw_vocab = {w: word_counter[w] for w in self.word2id.keys() if w in word_counter}

In [3]:
def cosine_similarity(e1, e2):
    """
    与えられた単語に最も似ている単語とcos類似度を返す関数

    :param embedding_path: str, 保存した埋め込み層のパラメータのパス
    :param word: str, 単語
    :param n: int
    :return out: str, 上位n個の類似単語とそのcos類似度
    """
    """
    embedding = torch.load(embedding_path)

    # 単語ベクトルを全て単位ベクトルにする
    norm = np.linalg.norm(embedding, ord=2, axis=1, keepdims=True)
    norm = np.where(norm==0, 1, norm) # 0で割ることを避ける
    embedding /= norm
    e = embedding[vocab.word2id[word]]
    """
    norm2 = np.linalg.norm(e2, ord=2)
    norm2 = np.where(norm2==0, 1, norm2) # 0で割ることを避ける
    e2 /= norm2
    
    norm1 = np.linalg.norm(e2, ord=2)
    norm1 = np.where(norm1==0, 1, norm1) # 0で割ることを避ける
    e1 /= norm1
    
    # 単語ベクトル同士のcos類似度を計算する
    cos_sim = np.dot(e1, np.transpose(e2)).reshape(-1,)
    #cos_sim = np.dot(e1, e2.reshape(-1, 1)).reshape(-1,)
    
    '''
    most_sim = np.argsort(cos_sim)[::-1][1:n+1] # 自分は除く
    most_sim_words = [vocab.id2word[_id] for _id in most_sim]
    top_cos_sim = cos_sim[most_sim]
    out = ", ".join([w+"({:.4f})".format(v) for w, v in zip(most_sim_words, top_cos_sim)])

    '''
    return cos_sim

In [4]:
# 特殊なトークンとそのIDは事前に定義しておきます。
PAD_TOKEN = '<PAD>' # あとで説明するpaddingに使います
UNK_TOKEN = '<UNK>' # 辞書にない単語は全てこのUNKトークンに置き換えます。(UNKの由来はunkownです)
PAD = 0 # <PAD>のID
UNK = 1 # <UNK>のID

In [5]:
# 辞書の初期化
word2id = {
    PAD_TOKEN: PAD,
    UNK_TOKEN: UNK,
}

# 辞書に含める単語の最低出現回数
# 今回はコーパスのサイズが小さいので、全ての単語を辞書に含めることにします
MIN_COUNT = 2

In [6]:
vocab = Vocab(word2id=word2id)
vocab.build_vocab(text, min_count=MIN_COUNT)

NameError: name 'text' is not defined

In [99]:
def sentence_to_ids(vocab, sen):
    """
    単語のリストをIDのリストに変換する関数

    :param vocab: class `Vocab` object
    :param sen: list of str, 文を分かち書きして得られた単語のリスト
    :return out: list of int, 単語IDのリスト
    """
    out = [vocab.word2id.get(word, UNK) for word in sen] # 辞書にない単語にはUNKのIDを割り振ります
    return out

In [84]:
# 日本語のテキストを単語IDに変換します。
id_text = [sentence_to_ids(vocab, sen) for sen in text]

In [85]:
print(text[2])
print(id_text[2])

['豚肉', 'の', '表面', 'を', 'キッチン', 'ペーパー', 'で', '丁寧', 'に', '水分', 'を', '拭き取っ', 'て', 'ください', '粗塩', 'を', '丁寧', 'に', 'まんべんなく', '豚肉', 'に', '擦り込み', 'ます', 'この', '分量', 'の', '肉', 'なら', '３', '５', 'ｇ', 'ぐらい', 'です', 'いい', '塩', 'ほど', '旨く', 'なる', 'って', 'いい', 'ます', 'ね', 'じっくり', '熟成', 'さ', 'せ', 'たい', 'から', '、', 'キッチン', 'ペーパー', 'で', '豚肉', 'を', '包み', '、', 'その', '上', 'から', 'ラップ', 'で', '包み', 'ます', 'そして', '冷蔵庫', 'で', '熟成', 'です', '１', '週間', '位', 'は', 'ドリップ', 'が', '出', 'て', 'き', 'ます', 'から', '、', '毎日', 'キッチン', 'ペーパー', 'と', 'ラップ', 'は', '交換', 'し', 'て', '下さい', 'この', '手間', 'が', '豚肉', 'を', '美味しく', 'し', 'ます', '豚肉', 'が', '小さけれ', 'ば', 'すぐ', 'に', '熟成', 'し', 'ます', 'が', '、', 'この', '状態', 'で', '約', '３', '週間', '経っ', 'て', 'い', 'ます', 'スライス', 'し', 'て', '熱湯', 'で', '茹で', 'て', 'ゆず', '味噌', 'で', 'いただき', 'まし', 'た']
[224, 7, 220, 2, 444, 326, 8, 980, 3, 155, 2, 2490, 6, 123, 4462, 2, 980, 3, 782, 224, 3, 4767, 12, 243, 164, 7, 100, 251, 45, 59, 497, 337, 28, 258, 31, 116, 6412, 71, 1331, 258, 12, 189, 647, 3360, 27, 63, 804, 38, 5, 444,

In [7]:
def pad_seq(seq, max_length):
    """Paddingを行う関数

    :param seq: list of int, 単語のインデックスのリスト
    :param max_length: int, バッチ内の系列の最大長
    :return seq: list of int, 単語のインデックスのリスト
    """
    seq += [PAD for i in range(max_length - len(seq))]
    return seq

In [9]:
embedding_path = "dataset/recipe_en/recipe_en_cbow_embedding_1000b_min_10_dim_500.pth"
embedding = torch.load(embedding_path)
print(embedding.shape)

(23245, 500)


In [148]:
# 単語ベクトルを全て単位ベクトルにする
#norm = np.linalg.norm(embedding, ord=2, axis=1, keepdims=True)
#norm = np.where(norm==0, 1, norm) # 0で割ることを避ける
#embedding /= norm
e1 = embedding[vocab.word2id['玄米']]
e2= embedding[vocab.word2id['白米']]

In [149]:
cosine_similarity(e1, e2)

array([-0.04884906], dtype=float32)

In [128]:
clusterVec = [embedding[0]]     # tracks sum of vectors in a cluster
clusterIdx = []    # array of index arrays. e.g. [[1, 3, 5], [2, 4, 6]]
ncluster = 0

In [129]:
# probablity to create a new table if new customer
# is not strongly "similar" to any existing table
pnew = 1.0/ (1 + ncluster)  
N = len(embedding)
#rands = random.rand(N)         # N rand variables sampled from U(0, 1)
print(N)

7432


In [143]:
v = embedding[0]
sim = cosine_similarity(v, clusterVec[0])
print(sim)

[-0.04192124]


In [131]:
 for i in range(N):
    maxSim = -float('inf')
    maxIdx = 0
    v = embedding[i]
    for j in range(ncluster):
        sim = cosine_similarity(v, clusterVec[j])
        if sim > maxSim:
            maxIdx = j
            maxSim = sim
    if maxSim < pnew:
        if random.random() < pnew:
            clusterVec.append(v)
            clusterIdx.append([i])
            ncluster += 1
            pnew = 1.0 / (1 + ncluster)
            continue
    clusterVec[maxIdx] = clusterVec[maxIdx] +v
    clusterIdx[maxIdx].append(i)

In [132]:
print(len(clusterIdx))

20


In [133]:
clusterIdx[1]

[961,
 1292,
 1376,
 1408,
 1527,
 1529,
 1530,
 1553,
 1574,
 1654,
 1677,
 1698,
 1822,
 1844,
 1849,
 1876,
 1885,
 1950,
 1954,
 2061,
 2119,
 2165,
 2208,
 2230,
 2303,
 2410,
 2418,
 2438,
 2519,
 2590,
 2657,
 2709,
 2721,
 2781,
 2783,
 2843,
 2854,
 2855,
 2945,
 2961,
 3075,
 3089,
 3138,
 3139,
 3202,
 3212,
 3237,
 3246,
 3275,
 3293,
 3365,
 3367,
 3412,
 3445,
 3472,
 3483,
 3509,
 3518,
 3527,
 3544,
 3582,
 3618,
 3620,
 3645,
 3659,
 3710,
 3716,
 3724,
 3758,
 3783,
 3821,
 3845,
 3863,
 3874,
 3886,
 3889,
 3909,
 3919,
 3980,
 3988,
 4007,
 4022,
 4038,
 4068,
 4072,
 4110,
 4146,
 4153,
 4276,
 4304,
 4325,
 4337,
 4407,
 4428,
 4491,
 4507,
 4516,
 4517,
 4539,
 4570,
 4596,
 4652,
 4660,
 4665,
 4681,
 4700,
 4704,
 4709,
 4718,
 4768,
 4818,
 4823,
 4843,
 4844,
 4853,
 4910,
 4916,
 4963,
 4970,
 4981,
 4982,
 5000,
 5012,
 5019,
 5025,
 5039,
 5048,
 5055,
 5060,
 5068,
 5070,
 5087,
 5091,
 5117,
 5123,
 5182,
 5190,
 5209,
 5213,
 5247,
 5280,
 5348,
 5381,


In [134]:
for i in range(0,len(clusterIdx)):
    print(len(clusterIdx[i]))

3859
233
222
219
238
232
213
251
206
217
210
187
220
224
200
156
140
85
99
21


In [137]:
for i in range(0, len(clusterIdx[1])):
    print(vocab.id2word[clusterIdx[1][i]])

きのこ
胸
ミンチ
カマ
シメジ
ぐるぐる
たけ
やっ
変更
ちゃん
重なら
固まる
裏面
捏ねる
この間
♪）
消毒
熱く
もっと
ティー
ふわっと
艶
反対
小分け
色づい
くっつい
煮立てる
つまん
雑煮
切れる
いただけ
バラバラ
抑え
サトイモ
湯気
ホウレンソウ
触っ
モチ
体
ロール
くっつく
F
ゆかり
スペース
アスパラガス
ライム
おろ
しっぽ
程よく
パルメザン
選ん
175
来る
敷きつめ
ミント
決め
ばら肉
マスカルポーネ
大蒜
デス
対
羽
オープン
化
わける
はね
白子
吸い物
%
乾か
香味
事前
ぐちゃぐちゃ
❀
ゲソ
入れ替え
黄金
マドレーヌ
⑨
ショップ
変える
突き
みこむ
対応
多
軟らか
ハーブソルト
各々
意外と
ーー
関節
ღ
まぜあわせる
合わさっ
身体
ブタ
折れる
製氷
いで
かえし
版
成分
パサ
ふやけ
食う
ツメ
紙カップ
お子さん
めし
毎日
|
たまに
素材
煮含める
鯵
点々
みんな
細切れ
】(
スライド
彩
剝
−
難い
コンビーフ
茹だっ
なか
取
840720
生臭く
喜び
ひび
タテ
詰め込む
ヤマサ
フワ
旗
撮っ
布き
ティッシュ
すりまぜる
仕上がっ
重箱
ガリガリ
太く
あんに
柑橘
ヌメリ
メッセージ
キャセロール
ブナ
スピード
本日
ポケット
薦め
薬
分厚く
ヤンニョム
メンチ
225
合い挽き
ほっぺた
つつき
ほしい
混ん
♬♩♫♪
ｱﾙﾐﾎｲﾙ
プルプル
揚がる
ハッシュドポテト
沈殿
ぽい
歯応え
ぐみ
焦げめ
すすぎ
(^。^)
目的
ぬるい
‐
少
ホイップカスタード
カルピス
本来
Ｕ
叩きつける
煮含め
おや
早速
間違え
クラム
不可
速度
保護
あんぱん
うれしい
既に
分から
白桃
田麩
にきび
日陰
難く
両足
マグネシウム
ヘモグロビン
カリカリベーコン
982138
粒状
）④
ホイール
実に
ｵｰﾌﾞﾝｼｰﾄ
!(^^)!
黄な粉
失礼
ﾌﾞﾘ
キット
サン
（③
調
ギリ
～＾＾
溢れ
軍手
壊さ
ﾍﾞﾗ
添っ
プッチーナ
炊き込む
押し麦
渡し
うま味


In [122]:
csvfile = pd.read_csv('dataset/Material/replaced.csv', encoding='utf-8')
file_processes = csvfile['processes'][0:100]

In [123]:
def data_tokenize(df):
    text=[]
    for recipe in df:
        word = tokenize(recipe)
        text.append(word)
    return text

In [124]:
data_tokenize(file_processes)

[['うどん',
  '・',
  '餅',
  '以外',
  'の',
  '材料',
  'を',
  '食べ',
  'やすい',
  '大き',
  'さ',
  'に',
  '切る',
  '鍋',
  'か',
  '土鍋',
  'に',
  '水',
  '・',
  '酒',
  '・',
  '醤油',
  '・',
  '粉末',
  'だし',
  'を',
  '加え',
  'て',
  '火',
  'にかけて',
  '煮',
  'たてる',
  '白菜',
  '・',
  '豚肉',
  'を',
  '交互',
  'に',
  '重ね',
  'て',
  '最後',
  'に',
  '白',
  '葱',
  'を',
  'のせ',
  '、',
  '他',
  'の',
  '材料',
  'も',
  '入れる',
  '材料',
  'に',
  '火',
  'が',
  '通れ',
  'ば',
  '出来上がり',
  '♪'],
 ['魚肉',
  'ソーセージ',
  'を',
  'すり',
  '下ろし',
  '、',
  'を',
  '全て',
  '入れ',
  '混ぜる',
  '卵',
  'を',
  '割り',
  '解し',
  '、',
  'を',
  '入れ',
  '混ぜ',
  '油',
  'を',
  '惹い',
  'た',
  'フライパン',
  'で',
  'そぼろ',
  '状',
  'に',
  '炒める',
  '。',
  '弁当',
  '箱',
  'に',
  'ご飯',
  'を',
  '敷き詰め',
  '、',
  '魚肉',
  'ソーセージ',
  'と',
  '卵',
  'を',
  '並べ',
  'て',
  '入れ',
  'て',
  '、',
  '今日',
  'の',
  'お',
  '弁当',
  'の',
  '出来上がり',
  '〜'],
 ['豚肉',
  'の',
  '表面',
  'を',
  'キッチン',
  'ペーパー',
  'で',
  '丁寧',
  'に',
  '水分',
  'を',
  '拭き取っ',
  'て',
  'ください',
  '粗塩',
  'を',
 